# Mas o que é o Clean Architecture?

É uma arquitetura de software proposta por Robert Cecil Martin (ou Uncle Bob, como é mais conhecido) que tem por objetivo padronizar e organizar o código desenvolvido, favorecer a sua reusabilidade, assim como independência de tecnologia.

<div style="text-align: center;">
  <a href="https://blog.cleancoder.com/uncle-bob/2012/08/13/the-clean-architecture.html">
    <img src="https://blog.cleancoder.com/uncle-bob/images/2012-08-13-the-clean-architecture/CleanArchitecture.jpg" alt="Clean Architecture" width="500"/>
  </a>
</div>

Os formatos de dados usados ​​em um círculo externo não devem ser usados ​​por um círculo interno, especialmente se esses formatos forem gerados por uma estrutura em um círculo externo. Não queremos que nada no círculo externo tenha impacto nos círculos internos.

# Configurando um ambiente VENV com python

```bash
# Criando um ambiente venv
$ /python -m venv venv

# Ativando o ambiente venv
$ /source venv/Scripts/activate
```

# Instalando o Pylint

```bash
# Instalando lib
$ /pip install pylint
```

# Criando arquivo de configuração para o Vscode

```json
{
  "python.linting.enable": true,
  "python.linting.pylintEnabled": true,
}
```

# Criando arquivo de configuração do pylint para modificar algumas regras
```bash
# Criando o arquivo de configuração do pylint
$ /pylint --generate-rcfile > .pylintrc
```

# Criando arquivo requirements.txt
```bash
# Criando arquivo requirements.txt
$ /pip freeze > requirements.txt

# Para instalar os pacotes atraves de um arquivo requirements.txt
$ /pip install -r requirements.txt
```

# Instalando o gerente de commits git
```bash
# Instalando o pre-commit
$ /pip install pre-commit
```

# Criando arquivo de configuração para o funcionamento do pre-commit .pre-commit-config.yaml

```yaml
  repos:
    - repo: local
      hooks:
        - id: pylint
          name: pylint
          entry: pylint
          language: system
          type: [python]
          args:
            [
              "-rn", # Only display message
              "-sn", # Don't display the score
              "--rcfile=.pylintrc", # Link to your config file
              "--load-plugins=pylint.extensions.docparams", # Load an extension
            ]
  - repo: local
    hooks:
      - id: requirements
        name: requirements
        entry: bash -c 'pip freeze > requirements.txt; git add requirements.txt'
        language: system
        pass_filenames: false
        stages: [commit]
```

# Criando os diretórios para iniciar a construção do projeto:

- Criar pasta src
  - Aqui vamos compilar todos os scripts python
- Criar pas init
  - Aqui vamos  criar scripts que inicianizam os arquivos dentro da pasta src
  - Criar um arquivo chamado schema.sql